In [67]:
import time
import json
import sys
import datetime
import os
sys.path.append('PriceRequest_UberAPI/')
from itertools import permutations
#from PriceRequest_UberAPI import uberpricing
#from messageProducer import genMessage

In [43]:
from kafka import KafkaClient, KeyedProducer, SimpleProducer

'''This is a program to send message to Kafka Topic. Start and End point is used as unique Key for the Keyed producer'''
kafka=KafkaClient("localhost:9092")
producer = KeyedProducer(kafka)
def genMessage(msg,key,topic):
    producer.send_messages(topic,key, msg)

In [53]:
from uber_rides.session import Session
from uber_rides.client import UberRidesClient

session = Session(server_token='DGdtuZb4i7zwENbYaVSPwuXLBtnzwkI2DigBmtmf')
def getPriceEstimate(key,start_lat,start_long,end_lat,end_long):
    client = UberRidesClient(session)
    p=client.get_price_estimates(start_lat,start_long,end_lat,end_long)
    k_lat_long=key+"|"+str(start_lat)+"|"+str(start_long)+"|"+str(end_lat)+"|"+str(end_long)
    value=p.json.get('prices')
    #value['key'] = k_lat_long
    return (value),str(k_lat_long)

In [45]:
#List of Locations
loc1=(37.787817,-122.396595)#Galvanize
loc2=(37.776908,-122.394825)#Caltrain
loc3=(37.810470, -122.476624)#
loc4=(37.775505, -122.446444)#USF
loc5=(37.810604,-122.409856)#Pier39
loc6=(37.687802, -122.470648)#Daly City
loc7=(37.737842, -122.431546)#GlenPark
loc8=(37.768719, -122.488666)#GoldenGatePark
loc9=(37.621206, -122.388759)#Airport
loc10=(37.692554, -122.432361)#San Bruno MOuntain State Park

location_dict={}
permutation_dict={}
location_dict.update({'A':loc1,
                      'B':loc2,
                      'C':loc3,
                      'D':loc4,
                      'E':loc5,
                      'F':loc6, 
                      'G':loc7, 
                      'H':loc8,
                      'G':loc9,
                      'I':loc10})

loc_list=[loc1,loc2,loc3,loc4,loc5,loc6,loc7,loc8,loc9,loc10]
topic="test"

#total_route=list(permutations(location_dict.itervalues(),2))
#ordered_route=sorted(total_route,key=(lambda x:x[1]))

total_route=list(permutations(location_dict,2))
ordered_route=sorted(total_route,key=(lambda x:x[0]))
for start,dest in ordered_route:
    key = start+'-'+dest
    permutation_dict[key] = [(location_dict[start],location_dict[dest])]

In [46]:
'''This is a job scheduler which sends a request to Uber API 
to get p
ricing Details and then send message to Kafka Topic.
'''

for route in permutation_dict:
    message,key=getPriceEstimate(route,permutation_dict[route][0][0][0],permutation_dict[route][0][0][1],permutation_dict[route][0][1][0],permutation_dict[route][0][1][1])
    genMessage(message,key,topic)
    

KeyboardInterrupt: 

In [64]:
import ast
from google.cloud import bigtable
from google.cloud import happybase
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\Users\Pradeep\Google Drive\Uber_Pricing_Pipelling\UBER-Price-Surge-fe496f50d763.json"
#client_credentials = json.load(open("<path to .json credentials>"))
project_id = 'uber-price-surge'
instance_id = 'uber-price' 
table_name = 'uber_price'
cluster = 'uber-price-cluster'
zone = 'us-east1-b'

def connection(project_id, instance_id ):
    client = bigtable.Client(project=project_id, admin=True)
    instance = client.instance(instance_id)
    connection = happybase.Connection(instance=instance)
    return connection

connection = connection(project_id, instance_id )
table = connection.table(table_name)

In [25]:
print('Inserting values into the {} table.'.format(table_name))

table.put(value[0], {column_name:json.dumps(value[1])})

Inserting values into the price_table table.


NameError: name 'value' is not defined

In [68]:
for route in permutation_dict:
    message,key=getPriceEstimate(route,permutation_dict[route][0][0][0],permutation_dict[route][0][0][1],permutation_dict[route][0][1][0],permutation_dict[route][0][1][1])
    
    for i in message:

        #val_dict = ast.literal_eval(i)
        key = key+ str(datetime.datetime.now()) 
        val_dict = i
        if (val_dict['low_estimate'] != None and  val_dict['high_estimate']!= None):
            dic_values={'distance': val_dict['distance'],
                    'low_estimate':val_dict['low_estimate'],
                    'high_estimate': val_dict['high_estimate'],
                    'price' : (val_dict['low_estimate']+val_dict['high_estimate'])/2,
                    'uber_type': val_dict['display_name']             
                   }
            column_family_name= dic_values['uber_type']
            column_name = '{fam}:price'.format(fam=column_family_name)
            table.put(key, {column_name:json.dumps(val_dict)})
            
 
    